In [39]:
import pandas as pd
import unidecode

# !pip install fuzzywuzzy
# !pip install python-Levenshtein

from fuzzywuzzy import fuzz


In [40]:
donors = pd.read_csv('past_donors_clean.csv', parse_dates=[1])

In [41]:
orgs = pd.read_excel('Organizations.xlsx')

In [54]:
orgs.head().T

#company names:princip filiales, name, courriel

def stdr_names(series_original):
    
    series = series_original.copy()
    series = series.str.upper()
    series = series.str.strip()

    series = series.replace(r'\s+', ' ', regex=True)
    series = series.str.replace(r'[^\w\s]+', '', regex=True)


    series = series.apply(lambda x: unidecode.unidecode(x))

    series = series.str.replace(' QUEBEC ', '')
    series = series.str.replace('CANADA', '')
    series = series.str.replace(' MONTREAL ', '')
    series = series.str.replace('MONTREAL ', '')
    series = series.str.replace(' MONTREAL', '')
    series = series.str.replace(' INC', '')
    series = series.str.replace(' INC ', '')
    series = series.str.replace(' CIE ', '')
    series = series.str.replace(' CIE', '')
    series = series.str.replace(' LTEE', '')
    series = series.str.replace('CORPORATION', '')
    series = series.str.replace('INTERNATIONAL', '')


    return series


In [43]:
donors = donors[ donors.company.notna() ]

In [44]:
orgs.NAME = stdr_names(orgs.NAME)

donors.company = donors.company.astype(str)
donors.company  = stdr_names(donors.company)

In [45]:
pd.merge(orgs.NAME.drop_duplicates(), donors.company.drop_duplicates(), left_on='NAME', right_on='company')

# 22 matches by exact matching

,NAME,company
0,ARCHAMBAULT,ARCHAMBAULT
1,AVERNA,AVERNA
2,CANADIAN TIRE,CANADIAN TIRE
3,CENTRES DENTAIRES LAPOINTE,CENTRES DENTAIRES LAPOINTE
4,CGI,CGI
5,CORBEC,CORBEC
6,DORFIN,DORFIN
7,ENERGIE CARDIO,ENERGIE CARDIO
8,FIERA CAPITAL,FIERA CAPITAL
9,FUTURE SHOP,FUTURE SHOP


In [46]:
# https://www.datacamp.com/tutorial/fuzzy-string-python

d = 0
j = 0
rows = []

for org in orgs.NAME.drop_duplicates().to_list():
    for donor in donors.company.drop_duplicates().to_list():
        Str1 = org
        Str2 = donor
        Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
        Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
        Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
        Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)

        row = [org, donor, Ratio, Partial_Ratio, Token_Sort_Ratio, Token_Set_Ratio]

        rows.append(row)

In [47]:
df_sim = pd.DataFrame(rows, columns = ['org', 'donor', 'Ratio', 'Partial_Ratio', 'Token_Sort_Ratio', 'Token_Set_Ratio'] )

df_sim.to_pickle('similarity_score.pkl')


In [142]:
score = 'Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(30)

,org,Ratio,donor
1309,KRUGER,100,KRUGER
578,AVERNA,100,AVERNA
1461,MAXI,100,MAXI
1218,IKEA,100,IKEA
974,ENERGIE CARDIO,100,ENERGIE CARDIO
612,BDO,100,BDO
843,CORBEC,100,CORBEC
731,CANADIAN TIRE,100,CANADIAN TIRE
1272,JEAN COUTU,100,JEAN COUTU
1060,FUTURE SHOP,100,FUTURE SHOP


In [143]:
score = 'Token_Set_Ratio'

df_ratios = df_sim.groupby('donor', as_index=False)[score].max()
df_token_set_ratio = df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False)

In [144]:
df_token_set_ratio[df_token_set_ratio.Token_Set_Ratio.ge(85)].to_clipboard()

In [145]:
score = 'Partial_Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

,org,Partial_Ratio,donor
1807,DORFIN,100,DORFIN
4280,VA,100,BOULEVARD MUSIQUE
4291,VA,100,SALVAIL ET CO
4290,VA,100,EMILIE VANDAL
4289,VA,100,SYLVAIN ROCHON DEPUTE DE RICHELIEU
4288,VA,100,FESTIVAL WESTERN DE ST TITE
4287,VA,100,METRO TRACY ALIMENTATION SYLVAIN BRIERE
4286,VA,100,CENTRE GERVAIS
4285,VA,100,AVANTI CINE VIDEO
4284,VA,100,E VAGUE


In [146]:


score = 'Token_Sort_Ratio'

df_ratios = df_sim.groupby('org', as_index=False)[score].max()
df_ratios.merge(df_sim[['org', 'donor', score]], how = 'inner').sort_values(score, ascending = False).head(20)

,org,Token_Sort_Ratio,donor
756,CENTRES DENTAIRES LAPOINTE,100,CENTRES DENTAIRES LAPOINTE
1820,SNC LAVALIN,100,SNC LAVALIN
1762,ROYAL LEPAGE,100,ROYAL LEPAGE
1272,JEAN COUTU,100,JEAN COUTU
1987,UPS,100,UPS
925,DORFIN,100,DORFIN
839,CORBEC,100,CORBEC
1477,MERCER,100,MERCER
1626,PFIZER,100,PFIZER
1451,MAXI,100,MAXI


In [48]:
th = 70

df_filt = df_sim.query(f''' Ratio > {th} |  Token_Sort_Ratio > {th}  | Token_Set_Ratio > {th}  ''' ).reset_index(drop=True)


In [49]:
df_filt.shape

(460, 6)

In [53]:
df_filt.sort_values(['Token_Set_Ratio', 'Ratio'], ascending=False).to_clipboard(index=None)